# Object Detection inference to submission.csv

# Imports

In [2]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')


## Env setup

In [3]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [4]:
from utils import label_map_util

from utils import visualization_utils as vis_util

/dk-jaden-tensorflow/models/research/object_detection/utils/visualization_utils.py:26: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  import matplotlib; matplotlib.use('Agg')  # pylint: disable=multiple-statements


# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [5]:
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = '/dk-jaden-tensorflow/models/research/object_detection/inference_497962' + '/frozen_inference_graph.pb'
#PATH_TO_FROZEN_GRAPH = '/dk-jaden-tensorflow/models/research/object_detection/ssd_model' + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('/dk-jaden-tensorflow/training_data', 'label_map.pbtxt')

## Load a (frozen) Tensorflow model into memory.

In [7]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [8]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
category_index

{1: {'id': 1, 'name': 'pedestrian'},
 2: {'id': 2, 'name': 'rider'},
 3: {'id': 3, 'name': 'vehicle'}}

## Helper code

In [9]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [10]:
import glob
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = '../../../testing_data/JPEGImages/All/*'

#TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, '{}.jpg'.format(i)) for i in range(1, 4) ]
TEST_IMAGE_PATHS =  sorted(glob.glob(str(PATH_TO_TEST_IMAGES_DIR)+'/*.jpg'))

print(TEST_IMAGE_PATHS)
# Size, in inches, of the output images.
IMAGE_SIZE = (36, 24)

['../../../creDa_test_public/JPEGImages/All/ctp_1.jpg', '../../../creDa_test_public/JPEGImages/All/ctp_10.jpg', '../../../creDa_test_public/JPEGImages/All/ctp_100.jpg', '../../../creDa_test_public/JPEGImages/All/ctp_1000.jpg', '../../../creDa_test_public/JPEGImages/All/ctp_1001.jpg', '../../../creDa_test_public/JPEGImages/All/ctp_1002.jpg', '../../../creDa_test_public/JPEGImages/All/ctp_1003.jpg', '../../../creDa_test_public/JPEGImages/All/ctp_1004.jpg', '../../../creDa_test_public/JPEGImages/All/ctp_1005.jpg', '../../../creDa_test_public/JPEGImages/All/ctp_1006.jpg', '../../../creDa_test_public/JPEGImages/All/ctp_1007.jpg', '../../../creDa_test_public/JPEGImages/All/ctp_1008.jpg', '../../../creDa_test_public/JPEGImages/All/ctp_1009.jpg', '../../../creDa_test_public/JPEGImages/All/ctp_101.jpg', '../../../creDa_test_public/JPEGImages/All/ctp_1010.jpg', '../../../creDa_test_public/JPEGImages/All/ctp_1011.jpg', '../../../creDa_test_public/JPEGImages/All/ctp_1012.jpg', '../../../creDa_test

In [11]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
      #print(image.shape[1],image.shape[2])

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: image})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      #print(output_dict['num_detections'])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.int64)
      #print(output_dict['detection_classes'])
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [12]:
import pandas as pd
submis = []
cnt=0
for image_path in TEST_IMAGE_PATHS:
  image_name = image_path.split('/')[-1]
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
  # Visualization of the results of a detection.
  '''vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)'''

  for output_score in output_dict.get('detection_scores'):
    if output_score > 0.5:
      i = list(output_dict.get('detection_scores')).index(output_score)
      submis.append([])
      submis[cnt].append(image_name)
      if list(output_dict['detection_classes'])[i] == 1:  #pedestrian
        submis[cnt].append(2)
      elif list(output_dict['detection_classes'])[i] == 2:  #rider
        submis[cnt].append(3)
      elif list(output_dict['detection_classes'])[i] == 3:  #vehicle
        submis[cnt].append(1)
      ymin, xmin, ymax, xmax = list(output_dict.get('detection_boxes'))[i]
      submis[cnt].append(int(xmin*image_np.shape[1]))
      submis[cnt].append(int(ymin*image_np.shape[0]))
      submis[cnt].append(int((xmax-xmin)*image_np.shape[1]))
      submis[cnt].append(int((ymax-ymin)*image_np.shape[0]))
      submis[cnt].append(output_score)
      print(submis[cnt])
      cnt=cnt+1
  #print('num of boxes: ',cnt)


['ctp_1.jpg', 3, 1498, 583, 145, 269, 0.9927517]
['ctp_1.jpg', 1, 385, 595, 339, 139, 0.9282555]
['ctp_1.jpg', 1, 820, 491, 336, 293, 0.8096503]
['ctp_1.jpg', 1, 225, 581, 209, 139, 0.6119754]
['ctp_10.jpg', 1, 523, 599, 387, 326, 0.99257106]
['ctp_100.jpg', 3, 1078, 191, 454, 635, 0.9907732]
['ctp_100.jpg', 3, 992, 302, 107, 137, 0.9549294]
['ctp_1000.jpg', 1, 274, 552, 429, 203, 0.99479717]
['ctp_1000.jpg', 3, 1692, 559, 206, 358, 0.96571666]
['ctp_1000.jpg', 3, 1539, 581, 95, 138, 0.75919944]
['ctp_1000.jpg', 3, 1114, 604, 68, 97, 0.7287514]
['ctp_1000.jpg', 1, 129, 588, 113, 64, 0.5226254]
['ctp_1001.jpg', 1, 113, 262, 392, 261, 0.9929876]
['ctp_1001.jpg', 1, 1015, 319, 96, 63, 0.92818683]
['ctp_1001.jpg', 1, 684, 316, 100, 65, 0.68890536]
['ctp_1001.jpg', 1, 833, 299, 150, 113, 0.55968404]
['ctp_1002.jpg', 1, 667, 554, 500, 408, 0.9778325]
['ctp_1002.jpg', 1, 1288, 557, 604, 430, 0.8476124]
['ctp_1003.jpg', 1, 611, 553, 533, 441, 0.9992692]
['ctp_1003.jpg', 3, 1417, 468, 482, 611,

['ctp_1039.jpg', 1, 1230, 364, 617, 519, 0.9998047]
['ctp_1039.jpg', 1, 722, 570, 366, 333, 0.9996841]
['ctp_1039.jpg', 1, 1075, 613, 109, 77, 0.8562069]
['ctp_1039.jpg', 3, 637, 602, 37, 54, 0.5499585]
['ctp_104.jpg', 1, 792, 292, 97, 59, 0.9973194]
['ctp_104.jpg', 1, 621, 295, 55, 37, 0.8751279]
['ctp_104.jpg', 1, 547, 287, 68, 45, 0.69255924]
['ctp_1040.jpg', 1, 783, 182, 230, 261, 0.99244446]
['ctp_1041.jpg', 1, 874, 607, 302, 233, 0.99440515]
['ctp_1041.jpg', 3, 1414, 612, 236, 377, 0.9007663]
['ctp_1041.jpg', 1, 1288, 622, 298, 244, 0.90038025]
['ctp_1042.jpg', 1, 1026, 258, 144, 122, 0.99170375]
['ctp_1043.jpg', 1, 512, 228, 530, 437, 0.999529]
['ctp_1043.jpg', 1, 37, 281, 382, 161, 0.9979634]
['ctp_1044.jpg', 1, 813, 274, 70, 44, 0.98363197]
['ctp_1044.jpg', 1, 925, 278, 66, 41, 0.96962637]
['ctp_1044.jpg', 1, 414, 259, 215, 126, 0.9639524]
['ctp_1045.jpg', 1, 1, 293, 316, 194, 0.97313964]
['ctp_1045.jpg', 1, 1066, 318, 126, 75, 0.9371163]
['ctp_1045.jpg', 1, 836, 300, 160, 116

['ctp_1081.jpg', 1, 729, 284, 64, 46, 0.9935644]
['ctp_1081.jpg', 1, 588, 279, 125, 91, 0.99222904]
['ctp_1081.jpg', 1, 825, 287, 55, 33, 0.9553788]
['ctp_1082.jpg', 1, 822, 272, 65, 46, 0.98865217]
['ctp_1082.jpg', 1, 925, 279, 64, 41, 0.94945914]
['ctp_1082.jpg', 1, 413, 261, 221, 127, 0.9400193]
['ctp_1083.jpg', 1, 823, 291, 64, 43, 0.9697897]
['ctp_1083.jpg', 1, 767, 297, 72, 49, 0.82940066]
['ctp_1083.jpg', 1, 925, 291, 68, 42, 0.82307756]
['ctp_1083.jpg', 1, 476, 287, 206, 116, 0.6906815]
['ctp_1084.jpg', 1, 1240, 298, 405, 275, 0.99821585]
['ctp_1084.jpg', 1, 1105, 287, 147, 83, 0.988514]
['ctp_1084.jpg', 1, 670, 285, 106, 79, 0.9781601]
['ctp_1084.jpg', 1, 803, 271, 149, 128, 0.96624804]
['ctp_1084.jpg', 1, 1021, 300, 76, 59, 0.9395737]
['ctp_1084.jpg', 1, 367, 292, 161, 97, 0.6376969]
['ctp_1084.jpg', 1, 80, 249, 273, 206, 0.6180048]
['ctp_1085.jpg', 1, 636, 253, 173, 100, 0.88444173]
['ctp_1085.jpg', 1, 460, 276, 160, 78, 0.8581793]
['ctp_1085.jpg', 3, 1252, 281, 61, 71, 0.63

['ctp_1119.jpg', 1, 390, 102, 312, 358, 0.9991936]
['ctp_1119.jpg', 1, 162, 255, 187, 145, 0.99415827]
['ctp_1119.jpg', 2, 5, 207, 147, 451, 0.5516861]
['ctp_112.jpg', 1, 718, 288, 63, 36, 0.9913988]
['ctp_112.jpg', 1, 911, 284, 82, 49, 0.98450863]
['ctp_112.jpg', 1, 858, 287, 67, 39, 0.9044264]
['ctp_1120.jpg', 1, 797, 586, 393, 332, 0.9986481]
['ctp_1120.jpg', 1, 1323, 589, 490, 313, 0.99840826]
['ctp_1120.jpg', 1, 1182, 608, 219, 155, 0.7050189]
['ctp_1121.jpg', 1, 662, 548, 531, 415, 0.9804586]
['ctp_1121.jpg', 1, 1286, 562, 605, 435, 0.96303576]
['ctp_1121.jpg', 3, 289, 470, 270, 556, 0.54994375]
['ctp_1122.jpg', 1, 749, 577, 333, 301, 0.9972007]
['ctp_1122.jpg', 1, 657, 593, 155, 116, 0.95017105]
['ctp_1122.jpg', 1, 1432, 563, 487, 376, 0.9261204]
['ctp_1122.jpg', 1, 1180, 618, 301, 175, 0.7778684]
['ctp_1122.jpg', 1, 207, 599, 95, 47, 0.7631239]
['ctp_1122.jpg', 1, 593, 599, 127, 84, 0.64573145]
['ctp_1122.jpg', 1, 426, 605, 90, 52, 0.6262972]
['ctp_1122.jpg', 1, 963, 511, 271, 

['ctp_1161.jpg', 1, 838, 322, 84, 58, 0.96858615]
['ctp_1161.jpg', 1, 289, 319, 143, 73, 0.9054904]
['ctp_1161.jpg', 1, 0, 303, 167, 374, 0.5620705]
['ctp_1162.jpg', 1, 375, 299, 164, 96, 0.898553]
['ctp_1162.jpg', 1, 1135, 299, 170, 124, 0.8458743]
['ctp_1163.jpg', 1, 4, 545, 279, 356, 0.99893695]
['ctp_1163.jpg', 1, 281, 579, 388, 181, 0.994858]
['ctp_1163.jpg', 1, 822, 521, 356, 269, 0.7750633]
['ctp_1163.jpg', 3, 1293, 590, 140, 235, 0.5305739]
['ctp_1164.jpg', 1, 996, 278, 916, 552, 0.9558498]
['ctp_1165.jpg', 3, 840, 442, 206, 403, 0.8091039]
['ctp_1166.jpg', 1, 848, 281, 81, 59, 0.9411361]
['ctp_1166.jpg', 3, 1029, 270, 55, 86, 0.91925335]
['ctp_1166.jpg', 3, 1399, 293, 85, 85, 0.84703314]
['ctp_1166.jpg', 1, 467, 279, 177, 70, 0.8100232]
['ctp_1166.jpg', 1, 695, 269, 133, 74, 0.7921825]
['ctp_1166.jpg', 3, 1609, 313, 83, 74, 0.5923585]
['ctp_1167.jpg', 1, 708, 212, 408, 362, 0.9943146]
['ctp_1167.jpg', 1, 576, 295, 93, 68, 0.9308402]
['ctp_1167.jpg', 1, 1600, 288, 124, 50, 0.92

...

['ctp_8.jpg', 1, 699, 253, 293, 227, 0.9934662]
['ctp_8.jpg', 1, 1316, 298, 246, 94, 0.931173]
['ctp_8.jpg', 1, 1234, 302, 99, 62, 0.8811625]
['ctp_8.jpg', 1, 1786, 311, 133, 73, 0.8447977]
['ctp_8.jpg', 1, 565, 283, 122, 91, 0.806041]
['ctp_8.jpg', 1, 262, 291, 135, 63, 0.76601666]
['ctp_8.jpg', 1, 202, 301, 136, 59, 0.61006266]
['ctp_8.jpg', 3, 1667, 300, 67, 85, 0.59948254]
['ctp_80.jpg', 1, 815, 268, 106, 68, 0.9853609]
['ctp_80.jpg', 1, 850, 264, 139, 79, 0.5570565]
['ctp_800.jpg', 1, 690, 587, 299, 261, 0.999806]
['ctp_800.jpg', 3, 1590, 571, 205, 288, 0.99672264]
['ctp_800.jpg', 1, 334, 596, 164, 132, 0.9863116]
['ctp_800.jpg', 1, 1008, 593, 256, 182, 0.7738087]
['ctp_800.jpg', 1, 603, 576, 161, 147, 0.74311894]
['ctp_800.jpg', 1, 102, 593, 179, 127, 0.53645176]
['ctp_801.jpg', 1, 44, 580, 521, 304, 0.9963246]
['ctp_801.jpg', 1, 842, 491, 245, 269, 0.9897092]
['ctp_801.jpg', 3, 1679, 600, 219, 394, 0.9812424]
['ctp_801.jpg', 3, 1213, 585, 169, 325, 0.9471737]
['ctp_801.jpg', 1, 

['ctp_839.jpg', 1, 824, 290, 57, 34, 0.99358773]
['ctp_839.jpg', 1, 745, 273, 63, 52, 0.9935022]
['ctp_839.jpg', 1, 610, 256, 127, 102, 0.9863979]
['ctp_84.jpg', 1, 1031, 253, 182, 134, 0.9894838]
['ctp_84.jpg', 1, 1207, 251, 145, 90, 0.7339985]
['ctp_840.jpg', 1, 598, 553, 553, 449, 0.9994886]
['ctp_840.jpg', 1, 1429, 548, 490, 479, 0.94746107]
['ctp_841.jpg', 1, 672, 601, 327, 285, 0.99987066]
['ctp_841.jpg', 3, 1485, 575, 202, 274, 0.9938964]
['ctp_841.jpg', 1, 467, 614, 141, 110, 0.9562726]
['ctp_841.jpg', 1, 547, 613, 173, 137, 0.93100303]
['ctp_841.jpg', 1, 25, 611, 372, 257, 0.7518928]
['ctp_841.jpg', 1, 992, 609, 262, 185, 0.62882847]
['ctp_842.jpg', 1, 459, 277, 160, 83, 0.8641915]
['ctp_842.jpg', 1, 678, 265, 129, 79, 0.8573567]
['ctp_842.jpg', 3, 1267, 283, 54, 68, 0.5440249]
['ctp_842.jpg', 2, 1421, 298, 52, 72, 0.5214647]
['ctp_843.jpg', 1, 1172, 302, 189, 111, 0.9828637]
['ctp_843.jpg', 3, 380, 291, 93, 143, 0.91960484]
['ctp_843.jpg', 1, 1084, 306, 110, 86, 0.81993234]
[

['ctp_878.jpg', 1, 634, 566, 624, 463, 0.9999362]
['ctp_878.jpg', 1, 1716, 693, 203, 342, 0.9025513]
['ctp_878.jpg', 1, 1194, 617, 237, 153, 0.66756064]
['ctp_878.jpg', 2, 1684, 620, 105, 163, 0.5433507]
['ctp_879.jpg', 1, 729, 256, 265, 229, 0.9995901]
['ctp_879.jpg', 1, 1412, 302, 78, 42, 0.99588495]
['ctp_879.jpg', 1, 928, 297, 77, 64, 0.8624979]
['ctp_879.jpg', 1, 692, 283, 93, 70, 0.84367687]
['ctp_879.jpg', 1, 3, 7, 638, 528, 0.7881729]
['ctp_879.jpg', 1, 626, 293, 74, 54, 0.76274115]
['ctp_88.jpg', 1, 606, 619, 358, 234, 0.99688977]
['ctp_88.jpg', 3, 474, 624, 108, 139, 0.98928785]
['ctp_88.jpg', 1, 173, 591, 249, 212, 0.6808346]
['ctp_88.jpg', 3, 80, 598, 92, 129, 0.65138096]
['ctp_88.jpg', 1, 1532, 679, 146, 94, 0.61105657]
['ctp_88.jpg', 1, 1843, 692, 72, 63, 0.53991735]
['ctp_880.jpg', 1, 663, 547, 548, 402, 0.9816135]
['ctp_880.jpg', 1, 1289, 554, 603, 429, 0.9334581]
['ctp_880.jpg', 3, 543, 545, 157, 328, 0.64228284]
['ctp_881.jpg', 1, 797, 291, 97, 58, 0.9607247]
['ctp_88

['ctp_921.jpg', 3, 1233, 621, 102, 144, 0.9298784]
['ctp_921.jpg', 1, 827, 525, 381, 281, 0.8032819]
['ctp_921.jpg', 1, 287, 537, 379, 246, 0.7768749]
['ctp_921.jpg', 1, 0, 535, 309, 384, 0.5893069]
['ctp_921.jpg', 3, 1364, 611, 133, 204, 0.5035855]
['ctp_922.jpg', 1, 211, 288, 218, 134, 0.99146444]
['ctp_922.jpg', 1, 785, 269, 166, 150, 0.98572993]
['ctp_922.jpg', 3, 69, 289, 119, 161, 0.9854512]
['ctp_922.jpg', 3, 958, 300, 37, 70, 0.97462976]
['ctp_922.jpg', 1, 1068, 255, 187, 163, 0.7721743]
['ctp_922.jpg', 3, 589, 292, 37, 75, 0.670492]
['ctp_922.jpg', 1, 3, 293, 145, 169, 0.6241109]
['ctp_923.jpg', 1, 18, 227, 1531, 635, 0.97464323]
['ctp_924.jpg', 1, 485, 579, 337, 286, 0.99996483]
['ctp_924.jpg', 1, 394, 606, 149, 99, 0.97866714]
['ctp_924.jpg', 1, 1209, 661, 156, 74, 0.76135194]
['ctp_925.jpg', 3, 851, 468, 198, 395, 0.5095756]
['ctp_926.jpg', 2, 1484, 550, 43, 86, 0.86463994]
['ctp_926.jpg', 1, 904, 550, 86, 57, 0.57767934]
['ctp_927.jpg', 1, 666, 551, 510, 410, 0.9743206]
['

['ctp_973.jpg', 1, 723, 617, 354, 286, 0.99994445]
['ctp_973.jpg', 3, 1280, 649, 165, 258, 0.9914586]
['ctp_973.jpg', 3, 532, 640, 69, 110, 0.97864795]
['ctp_973.jpg', 1, 348, 641, 186, 93, 0.94242513]
['ctp_973.jpg', 3, 1049, 674, 88, 151, 0.533694]
['ctp_974.jpg', 1, 636, 566, 624, 459, 0.99992406]
['ctp_974.jpg', 1, 351, 595, 338, 226, 0.9982582]
['ctp_974.jpg', 2, 1696, 623, 83, 155, 0.76149505]
['ctp_974.jpg', 1, 1215, 616, 239, 163, 0.7134101]
['ctp_974.jpg', 2, 1667, 626, 83, 151, 0.54369926]
['ctp_975.jpg', 1, 503, 573, 401, 361, 0.9984487]
['ctp_975.jpg', 3, 1093, 585, 183, 326, 0.9666077]
['ctp_975.jpg', 1, 962, 613, 202, 160, 0.9237186]
['ctp_975.jpg', 3, 321, 596, 64, 97, 0.89062726]
['ctp_975.jpg', 1, 1590, 518, 309, 233, 0.71405584]
['ctp_976.jpg', 1, 642, 567, 613, 464, 0.9999248]
['ctp_976.jpg', 1, 1178, 626, 206, 135, 0.9627867]
['ctp_976.jpg', 1, 1383, 573, 530, 389, 0.9143762]
['ctp_977.jpg', 1, 801, 608, 315, 270, 0.99969363]
['ctp_977.jpg', 1, 1244, 620, 311, 232, 

In [13]:
column_name = ['image_filename', 'label_id', 'x', 'y', 'w', 'h', 'confidence']
submis_df = pd.DataFrame(submis, columns=column_name)
#submis_df.to_csv('../../../creDa_test_public/JPEGImages/ssd_mobilenet_v1_coco_submission.csv', index=None)
#submis_df.to_csv('../../../creDa_test_public/JPEGImages/frcnn_irvac_submission.csv', index=None)
submis_df.to_csv('../../../creDa_test_public/JPEGImages/ssd_mobilenet_pets_submission.csv', index=None)